In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import sys
import os
from datetime import datetime, timedelta

import warnings
warnings.simplefilter('ignore')

from Local_lib.assets import *
from Local_lib.models import BlackScholesModel, HestonModel, SVJModel, SVCJModel
from Local_lib.period import Period
from Local_lib.volsurf import VolatilitySurface
from Local_lib.useful import bs_explicit_call

In [3]:
vs = VolatilitySurface.from_date(datetime(2019, 4, 1))
vs.calibrate()
#def bs_explicit_call(ttm, strike, spot, rate, SIGMA, dividends=0.):
#def iv(ttm, moneyness(spot/strike)):

bs_explicit_call(30/365, 4000, 4088.16, 0., vs.iv(30/365, 4088.16/4000)) ,\
bs_explicit_call(90/365, 4000, 4088.16, 0., vs.iv(90/365, 4088.16/4000))

(321.38682671745164, 553.854853530999)

In [4]:
vs = VolatilitySurface.from_date(datetime(2019, 10, 1))
vs.calibrate()
bs_explicit_call(30/365, 8000, 8367.51, 0., vs.iv(30/365, 8367.51/8000)), \
bs_explicit_call(90/365, 8000, 8367.51, 0., vs.iv(90/365, 8367.51/8000))

(1125.6539312410669, 1879.55334388164)

In [5]:
vs = VolatilitySurface.from_date(datetime(2020, 2, 1))
vs.calibrate()
bs_explicit_call(30/365, 10000, 9804.85, 0., vs.iv(30/365, 9804.85/10000)), \
bs_explicit_call(90/365, 10000, 9804.85, 0., vs.iv(90/365, 9804.85/10000))

(719.4803001961936, 1474.8100110892751)

In [3]:
t0 = datetime.strptime('20190401', '%Y%m%d')
t1 = datetime.strptime('20191001', '%Y%m%d')
t2 = datetime.strptime('20200201', '%Y%m%d')
t3 = datetime.strptime('20200701', '%Y%m%d')

In [9]:
#ttm correction
periods = [
    {
        'period': Period(t0, t1, 'Rising'),
        's0': 4088.16,
        'maturities': [30, 90],
        'strikes': [3000, 4000, 5000],
        'rate': 0,
    },
    {
        'period': Period(t1, t2, 'Calm'),
        's0': 8367.51,
        'maturities': [30, 90],
        'strikes': [7000, 8000, 9000],
        'rate': 0,
    },
    {
        'period': Period(t2, t3, 'Covid'),
        's0': 9804.85,
        'maturities': [30, 90],
        'strikes': [9000, 10000, 11000],
        'rate': 0,
    },
]

In [11]:
periods[0]["s0"]

4088.16

In [17]:
directory = 'Results'
os.makedirs(f"{directory}/Pricing", exist_ok=True)


# Pricing

In [35]:
rows = []

for _ in periods:
    
    period = _["period"]
    strikes = _["strikes"]
    maturities = _["maturities"]
    s0 = _["s0"]
    rate = _["rate"]
    
    vs = VolatilitySurface.from_date(period.t0, svi_caching=True)
    vs.calibrate()

    for strike in strikes:
        for maturity in maturities:
            title = f"{period.t0.strftime('%Y-%m-%d')}__{strike}__{maturity}"
            ttm = maturity / 365
            v0 = bs_explicit_call(ttm, strike, s0, rate, vs.iv(ttm, strike/s0))
            rows += [[title, v0]]
            
df = pd.DataFrame(rows, columns=['label', 'price'])
df.to_csv(f'{directory}/Pricing/__prices.csv')

In [67]:
df = pd.read_csv('Results/Pricing/__prices.csv')
df.to_clipboard(index=False, sep="&",float_format='{:.4f}'.format)

## Other models

In [2]:
def get_returns(period, interval):
    
    date_range = pd.date_range(period.t0 - timedelta(days=interval), period.t1)
    df = pd.read_csv("Results/BTCUSDT.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
    
    returns = np.log(df['Close']).diff()
    returns = returns.dropna()
    returns = returns.loc[date_range]
    
    return returns

In [5]:
black_scholes = BlackScholesModel()
heston = HestonModel()
svj = SVJModel()
svcj = SVCJModel()


In [47]:
periods = [
    {
        'period': Period(t0, t1, 'Rising'),
        's0': 4088.16,
        'maturities': [30, 90],
        'strikes': [3000, 4000, 5000],
        'rate': 0,
    },
    {
        'period': Period(t1, t2, 'Calm'),
        's0': 8367.51,
        'maturities': [30, 90],
        'strikes': [7000, 8000, 9000],
        'rate': 0,
    },
    {
        'period': Period(t2, t3, 'Covid'),
        's0': 9804.85,
        'maturities': [30, 90],
        'strikes': [9000, 10000, 11000],
        'rate': 0,
    },
]

In [54]:
np.random.seed(1337)

npaths = 100000
Prices = []
model = SVCJModel()
os.makedirs(f"{directory}/Pricing/{model.name}", exist_ok=True)

#    def paths(self, paths, period, s0, rate, npaths):

for _ in periods:
    period = _["period"]
    strikes = _["strikes"]
    maturities = _["maturities"]
    s0 = _["s0"]
    rate = _["rate"]
    paths = np.zeros((npaths, period.days))
    svcj.paths(paths,period,s0,rate,npaths)
    file_path = f'{directory}/Pricing/{model.name}/{period.name}_prices.csv'
    np.savetxt(file_path, paths, delimiter=',')


In [60]:
call_option_prices = []
os.makedirs(f"{directory}/Pricing/{model.name}/option_prices", exist_ok=True)
##put_option_prices = []
for _ in periods:

    call_payoffs = []
    ##put_payoffs = []

    period = _["period"]
    strikes = _["strikes"]
    maturities = _["maturities"]
    s0 = _["s0"]
    rate = _["rate"]

    paths = np.loadtxt(f'{directory}/Pricing/{model.name}/{period.name}_prices.csv', delimiter=',')

    for strike in strikes:
        for maturity in maturities:
            for path in paths:
                call_payoff = max(path[-1] - strike, 0)  
                ##put_payoff = max(strike - path[-1], 0)  
                call_payoffs.append(call_payoff)
                ##put_payoffs.append(put_payoff)
                
                # Discount 
                call_option_price = expected_call_payoff * np.exp(-rate * maturities[0] / 365)
                ##put_option_price = expected_put_payoff * np.exp(-rate * maturities[0] / 365)

            expected_call_payoff = np.mean(call_payoffs)
            ##expected_put_payoff = np.mean(put_payoffs)

            title = f"{period.t0.strftime('%Y-%m-%d')}__{strike}__{maturity}"
            call_option_prices += [[title, expected_call_payoff]]
            
df = pd.DataFrame(call_option_prices, columns=['label', 'price'])
df.to_csv(f"{directory}/Pricing/{model.name}/option_prices/call_prices.csv")    

#30 secondes

In [68]:
df = pd.read_csv(f"{directory}/Pricing/{model.name}/option_prices/call_prices.csv")
df.to_clipboard(index=False, sep="&",float_format='{:.4f}'.format)